# 我们将完成
1. 基准测试和性能分析框架
2. Flash Attention 
2. Triton 内核
3. 分布式数据并行训练
4. 优化器状态分片

# 前期准备工作

## 在windows上安装Ubuntu-22.04

由于triton在windows没有可用的版本，考虑到大家都是使用的windows系统的比较多，这里附一个安装教程，有windows系统的同学可以跳过

### 安装步骤

#### 1. 版本限制

我们需要在windows上安装wsl2，限制在Windows 10 2004 (19041) 及以上 或者 Windows 11 任意版本

#### 2.手动启用 WSL 相关功能（关键）

按住win键搜索**启用或者关闭windows功能**，找到并勾选 **于Linux的Windows子系统**和 **虚拟机平台**，点击确定


#### 3. 确认 CPU 虚拟化已开启

按住Ctrl + Shift + Esc打开任务管理器，性能 -> CPU ，右侧应显示：虚拟化：已启用

如果是“未启用”

需要进 BIOS / UEFI，开启：

Intel：Intel Virtualization Technology (VT-x)

AMD：SVM Mode

##### 4. 安装 Ubuntu

在windows搜索页面中输入CMD，并且以管理员身份运行

```bash
wsl --install -d Ubuntu
```

之后设置账号密码，密码在输入的时候是不可见的，看不到是正常的

要打开ubuntu的命令行只需要在cmd中上面一行的小三角中打开就行

##### 5. 安装相关的工具 pip uv 等

依次执行
先使用管理员身份打开CMD

输入
```bash
cd $env:USERPROFILE
@"
[wsl2]
networkingMode=mirrored
dnsTunneling=true
autoProxy=true
firewall=true
"@ | Out-File -Encoding UTF8 .wslconfig


wsl --shutdown     # 彻底关掉虚拟机
wsl                # 再进子系统
```

```bash
sudo apt update
sudo apt install python3-pip

sudo apt install python3.12-venv

source ~/myenv/bin/activate
pip install uv -i https://pypi.tuna.tsinghua.edu.cn/simple
sudo ln -s /home/kangkang/myenv/bin/uv /usr/local/bin/uv

echo 'export UV_INDEX_URL=https://pypi.tuna.tsinghua.edu.cn/simple' >> ~/.bashrc

source ~/.bashrc



```

#### 6. 设置vscode的远程连接

打开vscode ，先搜索插件wsl。安装完成后打开左侧一列的远程连接就可以连接到wsl的ubuntu系统中


运行在CS336-Chinese-co-construction\coursework\Assignment2_System下
```bash
uv venv 
cd cs336_basics
uv run
```



# 1. 基准测试和性能分析

编写一个脚本，用于对模型中的前向和反向传递执行基本的端到端基准测试。具体而言，脚本应支持以下功能
- 给定超参数（例如，层数），初始化一个模型。
- 生成一批随机数据。
- 在开始计时前，先运行 个预热步骤；随后，测量执行 个步骤所需的时间（根据参数设置，可仅进行正向传播，或同时包含正向与反向传播）。
- 每个步骤后调用 torch.cuda.synchronize()

## 首先检查GPU是否可用

这边通过pyproject.toml下载的torch是cpu版本
如果要安装GPU版本：
```bash
uv pip uninstall torch 
uv pip install torch --index-url https://download.pytorch.org/whl/cu121
```
cu121是我的cuda版本为12.1，可以改为自己的cuda版本

In [1]:
import torch, ctypes, os
print('torch:', torch.__version__)
print('cuda :', torch.version.cuda)
try:
    ctypes.CDLL('libcupti.so')
    print('CUPTI: OK')
except:
    print('CUPTI: NOT FOUND')

torch: 2.5.1+cu121
cuda : 12.1
CUPTI: NOT FOUND


In [2]:
import torch
import time
from typing import Callable

import math
import time
import torch.nn.functional as F

from cs336_basics.model import BasicsTransformerLM 
from cs336_basics.optimizer import get_cosine_lr
from cs336_basics.optimizer import AdamW
from cs336_basics.data import get_batch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("No CUDA GPU detected")

print("MPS（苹果的MPS） available:", hasattr(torch.backends, "mps") and torch.backends.mps.is_available())


Torch version: 2.5.1+cu121
CUDA available: True
CUDA device count: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4060 Laptop GPU
MPS（苹果的MPS） available: False


In [3]:
def mean(values: list[float]) -> float:
    if not values:
        raise ValueError("mean() requires at least one value")
    return sum(values) / len(values)

def benchmark(description: str, run: Callable, num_warmups: int = 1, num_trials: int = 3):
    """Benchmark `func` by running it `num_trials`, and return all the times."""
    # 热身：第一次运行可能较慢,因为要编译和缓存
    # 我们将多次要运行内核，因为重要的是稳态的运行时间。
    for _ in range(num_warmups):
        run()
    if torch.cuda.is_available():
        torch.cuda.synchronize()  # 等待 CUDA 线程完成（非常重要！）
    print('现在真正计时!')
    times: list[float] = [] # @inspect times, @inspect description
    for trial in range(num_trials):  # 多次重复
        start_time = time.time()
        run()  # 实际执行计算
        if torch.cuda.is_available():
            torch.cuda.synchronize()  # 等待 CUDA 线程 完成同步
        end_time = time.time()
        times.append((end_time - start_time) * 1000) # @inspect times
    mean_time = mean(times) # 多次测量取平均
    return mean_time

现在来测试一下sleep函数


In [4]:
benchmark("sleep", lambda : time.sleep(50 / 1000))

现在真正计时!


50.16009012858073

我们来实测一下transformer

In [5]:
vocab_size = 50_000
context_length = 128
batch_size = 32

d_model = 512
num_layers = 6
num_heads = 8
d_ff = 2048
rope_theta = 10000.0

max_lr = 3e-4
min_lr = 3e-5
warmup_iters = 200
cosine_cycle_iters = 10_000
num_steps = 10
device = "cuda" if torch.cuda.is_available() else "cpu"

model = BasicsTransformerLM(
    vocab_size=vocab_size,
    context_length=context_length,
    d_model=d_model,
    num_layers=num_layers,
    num_heads=num_heads,
    d_ff=d_ff,
    rope_theta=rope_theta,
).to(device)

optimizer = AdamW(
    model.parameters(),
    lr=max_lr,
    weight_decay=0.1,
)



def lm_loss(logits: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
    # logits: (B, T, V)
    # targets: (B, T)
    B, T, V = logits.shape
    return F.cross_entropy(
        logits.view(B * T, V),
        targets.view(B * T),
    )
import numpy as np

# 假设是 token 序列
dataset = np.arange(1000, dtype=np.int64)
x, y = get_batch(
    dataset=dataset,
    batch_size=4,
    context_length=8,
    device="cuda"
)

print(x.shape)  # (4, 8)
print(y.shape)  # (4, 8)
print(x)
print(y)


model.train()
print(f'设备：{device}')


torch.Size([4, 8])
torch.Size([4, 8])
tensor([[186, 187, 188, 189, 190, 191, 192, 193],
        [673, 674, 675, 676, 677, 678, 679, 680],
        [818, 819, 820, 821, 822, 823, 824, 825],
        [810, 811, 812, 813, 814, 815, 816, 817]], device='cuda:0')
tensor([[187, 188, 189, 190, 191, 192, 193, 194],
        [674, 675, 676, 677, 678, 679, 680, 681],
        [819, 820, 821, 822, 823, 824, 825, 826],
        [811, 812, 813, 814, 815, 816, 817, 818]], device='cuda:0')
设备：cuda


In [6]:
def run_model(num_steps = 1):
    for it in range(num_steps):
        print(f'{it}/{num_steps}')
        # 1️更新学习率（每 step）
        lr = get_cosine_lr(
            it=it,
            max_learning_rate=max_lr,
            min_learning_rate=min_lr,
            warmup_iters=warmup_iters,
            cosine_cycle_iters=cosine_cycle_iters,
        )
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr

        # 2取 batch
        x, y = get_batch(
            dataset=dataset,
            batch_size=batch_size,
            context_length=context_length,
            device=device,
        )
        print(x)

        # 3前向
        logits = model(x)

        # 4loss
        loss = lm_loss(logits, y)

        # 5反向
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        
        # （可选）梯度裁剪（强烈推荐）
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 6更新参数
        optimizer.step()

    # 7日志
    if it % 100 == 0:
        print(
            f"step {it:6d} | "
            f"loss {loss.item():.4f} | "
            f"lr {lr:.2e}"
        )


benchmark(description = 'transformer模型的基准测试', run =  run_model, num_warmups = 1, num_trials = 1)

0/1
tensor([[482, 483, 484,  ..., 607, 608, 609],
        [645, 646, 647,  ..., 770, 771, 772],
        [419, 420, 421,  ..., 544, 545, 546],
        ...,
        [ 85,  86,  87,  ..., 210, 211, 212],
        [614, 615, 616,  ..., 739, 740, 741],
        [740, 741, 742,  ..., 865, 866, 867]], device='cuda:0')
step      0 | loss 10.8311 | lr 0.00e+00
现在真正计时!
0/1
tensor([[ 53,  54,  55,  ..., 178, 179, 180],
        [374, 375, 376,  ..., 499, 500, 501],
        [536, 537, 538,  ..., 661, 662, 663],
        ...,
        [335, 336, 337,  ..., 460, 461, 462],
        [535, 536, 537,  ..., 660, 661, 662],
        [457, 458, 459,  ..., 582, 583, 584]], device='cuda:0')
step      0 | loss 10.8309 | lr 0.00e+00


442.5199031829834

## 推理：

In [7]:
model.eval()

with torch.no_grad():
    # 用 get_batch 拿一个 batch
    x, _ = get_batch(
        dataset=dataset,
        batch_size=1,
        context_length= 1,
        device=device,
    )

    # 取第一个样本作为 prompt
    prompt = x # shape: (context_length,)
    # 调用模型自带的 generate
    generated = model.generate(
        x=prompt,
        max_new_tokens=5,
        temperature=1.0,
        top_k=50,
        eos_token_id=None,
    )

    print(generated.shape)  # (<=50,)
    print(generated)


torch.Size([1, 5])
tensor([[19979, 43534,  7514, 44081, 28613]], device='cuda:0')


### 性能分析工具

In [8]:
from torch.profiler import ProfilerActivity
import os
def profile(description: str, run: Callable, num_warmups: int = 1, with_stack: bool = False):
    # 预热
    for _ in range(num_warmups):
        run()
    if torch.cuda.is_available():
        print("正在使用cuda")
        torch.cuda.synchronize()  # 等待CUDA线程结束
    else:
        print('正在使用cpu')
    # 使用性能分析器运行代码
    
    with torch.profiler.profile(
            activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
            # 输出堆栈跟踪以进行可视化
            with_stack=with_stack,
            #  需要导出堆栈跟踪以进行可视化
            experimental_config=torch._C._profiler._ExperimentalConfig(verbose=True)) as prof:
        run()
        if torch.cuda.is_available():
            torch.cuda.synchronize()  # 等待CUDA线程结束
    # 打印表格
    table = prof.key_averages().table(
        sort_by="self_cuda_time_total",
        max_name_column_width=80,
        row_limit=10,
        top_level_events_only=False) 
    #text(f"## {description}")
    #text(table, verbatim=True)
    # Write stack trace visualization
    if with_stack:
        os.makedirs("var", exist_ok=True)
        text_path = f"var/stacks_{description}.txt"
        svg_path = f"var/stacks_{description}.svg"
        prof.export_stacks(text_path, "self_cuda_time_total")
    return table
tabel = profile(description ='transformer' , run = run_model, num_warmups = 3, with_stack = True)
print(tabel)

0/1
tensor([[133, 134, 135,  ..., 258, 259, 260],
        [798, 799, 800,  ..., 923, 924, 925],
        [ 37,  38,  39,  ..., 162, 163, 164],
        ...,
        [771, 772, 773,  ..., 896, 897, 898],
        [517, 518, 519,  ..., 642, 643, 644],
        [565, 566, 567,  ..., 690, 691, 692]], device='cuda:0')
step      0 | loss 10.8347 | lr 0.00e+00
0/1
tensor([[754, 755, 756,  ..., 879, 880, 881],
        [762, 763, 764,  ..., 887, 888, 889],
        [ 84,  85,  86,  ..., 209, 210, 211],
        ...,
        [648, 649, 650,  ..., 773, 774, 775],
        [610, 611, 612,  ..., 735, 736, 737],
        [678, 679, 680,  ..., 803, 804, 805]], device='cuda:0')
step      0 | loss 10.8314 | lr 0.00e+00
0/1
tensor([[803, 804, 805,  ..., 928, 929, 930],
        [104, 105, 106,  ..., 229, 230, 231],
        [537, 538, 539,  ..., 662, 663, 664],
        ...,
        [756, 757, 758,  ..., 881, 882, 883],
        [276, 277, 278,  ..., 401, 402, 403],
        [552, 553, 554,  ..., 677, 678, 679]], de

# Flash Attention 

我们将在这里实现一个flash attention，我们需要实现两个部分，一个是分块，第二个是数值稳定softmax的方式，思想上非常接近 FlashAttention v1，具体可以看第六章：GPU和GPU相关的优化，有详细介绍


### **FlashAttention 中「分块（blocking）」与「数值稳定 softmax」为什么能带来加速**


### 一、加速原理

FlashAttention 的加速**不是因为算得更少 FLOPs**，而是因为：

> **极大减少了显存读写（memory I/O），并将计算变成“流式（streaming）”**

核心公式仍然是论文中的公式：

$$
\text{softmax}(QK^T)V
$$

但**计算路径完全改变**。

####  标准 Attention 的真实成本

以 sequence length = (N)，head dim = (d)：

| 操作      | FLOPs      | 显存占用          |
| ------- | ---------- | ------------- |
| QKᵀ     | (O(N^2 d)) | **存 N×N**     |
| softmax | (O(N^2))   | **读 + 写 N×N** |
| AV      | (O(N^2 d)) | 读 N×N         |

**关键瓶颈不是算力，而是显存带宽**

在 GPU 上：N×N的attention matrix **必须写入 HBM**，* 然后再 **从 HBM 读回** 做 softmax，大模型场景下 **显存访问 >> 计算时间**。

#### GPU 性能的现实约束

GPU 有三种“速度等级”存储：

| 存储            | 速度 | 容量 |
| ------------- | -- | -- |
| Register      | 极快 | 极小 |
| Shared Memory | 快  | 小  |
| HBM (显存)      | 慢  | 很大 |

标准 Attention 的问题是：

> **N×N attention matrix 太大，只能进 HBM**

#### 分块（Blocking）为什么能加速

**分块的本质就是让数据“活在片上”**

FlashAttention 将 Attention 重写为：

```text
for each Q_block:
    for each K/V_block:
        计算局部 QK^T
        立刻参与 softmax
        立刻乘 V
```

核心变化是不再生成完整的 QKᵀ，不再写入 N×N matrix，局部结果 **只存在寄存器 / shared memory**。从**全文总结变成了走一步看一步**。

#### 分块前 vs 分块后（显存行为对比）

**标准 Attention**

```text
HBM:
  写 QK^T (N×N)
  读 QK^T
  写 softmax(QK^T)
  读 softmax(QK^T)
```

**HBM I/O = O(N²)**

---

**FlashAttention（分块）**

```text
HBM:
  读 Q_block
  读 K_block, V_block
  写 O_block
```

**HBM I/O = O(N·d)**

**这是数量级的下降**

---

### 为什么“只对 K/V 分块也有效”

分块有两种实现：

* Q 不分块
* Q 分块（论文 / CUDA 版）

两者都满足：

> **K/V 被流式读取，attention matrix 不落地**

Q 分块是为了，提高 occupancy，并且控制寄存器使用，适配 CUDA block 结构。

---

#### 二、在线 softmax 的加速原理”

**标准 softmax 的结构问题**

标准 softmax：

```python
S = QK^T
P = exp(S) / sum(exp(S))
```

问题在于softmax **必须看完整一行**，无法流式，依赖全量 S。

**这强制我们先存下整个 QKᵀ**


**running max / running sum 的革命性意义**

FlashAttention 使用：

$$
\begin{aligned}
m_i &= \max_j S_{ij} \
l_i &= \sum_j e^{S_{ij} - m_i}
\end{aligned}
$$

并改写为 **可递推形式**：

$$
m_i^{(t)} = \max(m_i^{(t-1)}, \max S_{ij}^{(t)})
$$

$$
l_i^{(t)} =
l_i^{(t-1)} e^{m_i^{(t-1)} - m_i^{(t)}} +
\sum e^{S_{ij}^{(t)} - m_i^{(t)}}
$$


这一步“决定了 FlashAttention 能不能存在”，它带来了三件**决定性好处**，将softmax 变成 **在线算法**。

再这个算法中我们不需要完整 S，每来一个 block 就能更新，天然适配 streaming。

**softmax 与 AV 可以融合**

标准 Attention：

```text
QK^T → softmax → @V
```

FlashAttention：

```text
(QK^T → exp → sum → @V)  全部在 block 内完成
```

**算子融合（kernel fusion）成为可能**


**backward 也不需要 attention matrix**

由于 forward 保存了：

* `L = logsumexp`
* `O = softmax @ V`

backward 可写成：

$$
dS_{ij} = P_{ij} (dP_{ij} - D_i)
$$

**反向同样 O(N) 显存**

> FlashAttention 的加速不是减少计算，而是通过 **分块 + 在线 softmax**，
> **避免了 N×N attention matrix 的显存读写**，
> 将 attention 计算变成一个 **完全流式、可融合的过程**，
> 从而把瓶颈从显存带宽转回到算力。




In [9]:
import torch

class FlashAttentionAutograd(torch.autograd.Function):
    """
    FlashAttention（forward + backward）

    输入：
        Q: (B, Nq, d)
        K: (B, Nk, d)
        V: (B, Nk, d)

    输出：
        O: (B, Nq, d)
    """

    @staticmethod
    def forward(ctx, Q, K, V):
        B, Nq, d = Q.shape
        Nk = K.shape[1]

        # -----------------------------
        # block size
        # -----------------------------
        Bq = 64  # Query block 的尺寸
        Bk = 64  # Key / Value block 的尺寸

        # 我们将KV矩阵分成一个个小矩阵

        Tq = Nq // Bq  # Q block 数
        Tk = Nk // Bk  # K/V block 数

        scale = d ** -0.5

        # -----------------------------
        # 输出与中间量
        # -----------------------------
        O = torch.zeros_like(Q)              # Attention 输出
        L = torch.zeros(B, Nq, device=Q.device)  # log-sum-exp（给 backward 用）

        # =========================================================
        # 对 batch 维度显式循环
        # =========================================================
        for b in range(B):
            Q_b = Q[b]  # (Nq, d)
            K_b = K[b]  # (Nk, d)
            V_b = V[b]  # (Nk, d)

            # =====================================================
            # 外层循环：Query block
            # =====================================================
            for i in range(Tq):
                q_i = Q_b[i * Bq:(i + 1) * Bq]  # (Bq, d)

                # -----------------------------
                # FlashAttention 核心状态
                # -----------------------------
                m = torch.full((Bq, 1), -float("inf"), device=Q.device)  # running max
                l = torch.zeros((Bq, 1), device=Q.device)               # running sum
                O_acc = torch.zeros((Bq, d), device=Q.device)           # 输出累积（未归一化）

                # =================================================
                # 内层循环：Key / Value block
                # =================================================
                for j in range(Tk):
                    k_j = K_b[j * Bk:(j + 1) * Bk]  # (Bk, d)
                    v_j = V_b[j * Bk:(j + 1) * Bk]  # (Bk, d)

                    # --------------------------------------------
                    # 计算局部 attention score
                    # S_ij = Q_i @ K_j^T / sqrt(d)
                    # --------------------------------------------
                    S = q_i @ k_j.T * scale  # (Bq, Bk)

                    # 当前 block 内，每一行的最大值
                    row_max = S.max(dim=1, keepdim=True).values  # (Bq, 1)

                    # 更新 running max
                    m_new = torch.maximum(m, row_max)

                    # --------------------------------------------
                    # 数值稳定 softmax（FlashAttention 关键）
                    # --------------------------------------------
                    # 旧 block 贡献修正
                    exp_old = torch.exp(m - m_new) * l

                    # 当前 block 的 exp
                    P = torch.exp(S - m_new)

                    # 更新分母
                    l = exp_old + P.sum(dim=1, keepdim=True)

                    # 更新未归一化输出
                    O_acc = (
                        torch.exp(m - m_new) * O_acc +
                        P @ v_j
                    )

                    # 写回 running max
                    m = m_new

                # =================================================
                # block 内 softmax 归一化
                # =================================================
                O_i = O_acc / l  # (Bq, d)

                # log-sum-exp：L = m + log(l)
                L_i = m.squeeze(1) + torch.log(l.squeeze(1))

                # 写回全局输出
                O[b, i * Bq:(i + 1) * Bq] = O_i
                L[b, i * Bq:(i + 1) * Bq] = L_i

        # 保存 backward 所需变量
        ctx.save_for_backward(Q, K, V, O, L)
        ctx.scale = scale
        ctx.Bq = Bq
        ctx.Bk = Bk

        return O
    
    @staticmethod
    def backward(ctx, dO):
        Q, K, V, O, L = ctx.saved_tensors
        scale = ctx.scale
        Bq = ctx.Bq
        Bk = ctx.Bk

        B, Nq, d = Q.shape
        Nk = K.shape[1]

        Tq = Nq // Bq
        Tk = Nk // Bk

        # --------------------------------------------
        # FlashAttention backward 中的重要中间量
        # D_i = sum_j O_ij * dO_ij
        # --------------------------------------------
        D = torch.sum(O * dO, dim=-1)  # (B, Nq)

        dQ = torch.zeros_like(Q)
        dK = torch.zeros_like(K)
        dV = torch.zeros_like(V)

        for b in range(B):
            for i in range(Tq):
                q_i = Q[b, i * Bq:(i + 1) * Bq]
                dO_i = dO[b, i * Bq:(i + 1) * Bq]
                L_i = L[b, i * Bq:(i + 1) * Bq]
                D_i = D[b, i * Bq:(i + 1) * Bq]

                for j in range(Tk):
                    k_j = K[b, j * Bk:(j + 1) * Bk]
                    v_j = V[b, j * Bk:(j + 1) * Bk]

                    # --------------------------------------------
                    # 重算局部 attention 概率 P_ij
                    # P_ij = exp(S_ij - L_i)
                    # --------------------------------------------
                    S = q_i @ k_j.T * scale
                    P = torch.exp(S - L_i.unsqueeze(1))

                    # -------- dV --------
                    dV[b, j * Bk:(j + 1) * Bk] += P.T @ dO_i

                    # -------- dS --------
                    dP = dO_i @ v_j.T
                    dS = P * (dP - D_i.unsqueeze(1))

                    # -------- dQ --------
                    dQ[b, i * Bq:(i + 1) * Bq] += dS @ k_j * scale

                    # -------- dK --------
                    dK[b, j * Bk:(j + 1) * Bk] += dS.T @ q_i * scale

        return dQ, dK, dV


In [10]:
def test_flash_attention(B=1, N=1024, d=64, device="cuda"):
    torch.manual_seed(0)

    Q = torch.randn(B, N, d, device=device, requires_grad=True)
    K = torch.randn(B, N, d, device=device, requires_grad=True)
    V = torch.randn(B, N, d, device=device, requires_grad=True)

    def run():
        O = FlashAttentionAutograd.apply(Q, K, V)
        #loss = O.sum()
        #loss.backward()

    t = benchmark(description= 'test_flash_attention',run = run)
    t1 = profile(description ='test_flash_attention' , run = run, num_warmups = 3, with_stack = True)
    
    print(f"[FlashAttention] B={B}, N={N}, d={d}, time={t*1000:.3f} ms")
    print(t1)



def naive_attention(Q, K, V):
    d = Q.size(-1)
    scores = Q @ K.transpose(-2, -1) / (d ** 0.5)
    P = torch.softmax(scores, dim=-1)
    return P @ V
def test_naive_attention(B=1, N=1024, d=64, device="cuda"):
    torch.manual_seed(0)

    Q = torch.randn(B, N, d, device=device, requires_grad=True)
    K = torch.randn(B, N, d, device=device, requires_grad=True)
    V = torch.randn(B, N, d, device=device, requires_grad=True)

    def run():
        O = naive_attention(Q, K, V)
        #loss = O.sum()
        #loss.backward()

    t = benchmark(description= 'test_attention',run = run)
    t1 = profile(description ='test_attention' , run = run, num_warmups = 3, with_stack = True)
    print(f"[NaiveAttention]  B={B}, N={N}, d={d}, time={t*1000:.3f} ms")
    print(t1)


device = "cuda"

for N in [1024]:
    test_flash_attention(B=1, N=N, d=64, device=device)
    test_naive_attention(B=1, N=N, d=64, device=device)
    print("-" * 50)


现在真正计时!
正在使用cuda
[FlashAttention] B=1, N=1024, d=64, time=63135.068 ms
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                           FlashAttentionAutograd        23.36%      20.047ms        99.99%      85.825ms      85.825ms             1  
                                 aten::zeros_like         0.01%      10.541us         2.81%       2.415ms       2.415ms             1  
                                 aten::empty_like         0.01%       8.343us         2.73%       2.340ms       2.340ms             1  
                              aten::empty_strided         2.72%       2.331ms         2.72%       2.331ms       2

# 2. Triton 内核实现flash attention

Triton 是一个用 Python 写 GPU kernel 的 DSL，本质上是“显式控制数据分块与内存层级的编程模型”，它不是NumPy，也不是高层 PyTorch API，它是一个 CUDA kernel 的“可读版本”。

In [11]:
# from sympy import Q
import torch
import triton
import triton.language as tl

@triton.jit
def flash_attention_fwd_kernel(
    # -------- 全局内存指针 --------
    Q_ptr, K_ptr, V_ptr,           # 输入 Q, K, V
    O_ptr, L_ptr,                  # 输出 O, logsumexp L

    # -------- stride 信息（用于 block ptr）--------
    stride_qb, stride_qn, stride_qd,
    stride_kb, stride_kn, stride_kd,
    stride_vb, stride_vn, stride_vd,
    stride_ob, stride_on, stride_od,
    stride_lb, stride_ln,

    # -------- 序列长度 --------
    Nq, Nk,

    # -------- scale --------
    scale,

    # -------- 编译期常量 --------
    D: tl.constexpr,
    Q_BLOCK: tl.constexpr,
    K_BLOCK: tl.constexpr,
    is_causal: tl.constexpr,
):
    """
    每个 Triton program 负责：
        - 一个 batch
        - 一个 Query block（Q_BLOCK 行）
    """

    # ------------------------------------------------
    # Triton 并行索引
    # ------------------------------------------------
    q_block_id = tl.program_id(0)     # 第几个 Q block
    batch_id   = tl.program_id(1)     # 第几个 batch

    # =================================================
    # 构造 block pointer（这是 Triton 的关键）
    # =================================================
    Q_block_ptr = tl.make_block_ptr(
        base=Q_ptr + batch_id * stride_qb,
        shape=(Nq, D),
        strides=(stride_qn, stride_qd),
        offsets=(q_block_id * Q_BLOCK, 0),
        block_shape=(Q_BLOCK, D),
        order=(1, 0),
    )

    # ⚠️ K / V 从第 0 行开始，后面在 loop 中 advance
    K_block_ptr = tl.make_block_ptr(
        base=K_ptr + batch_id * stride_kb,
        shape=(Nk, D),
        strides=(stride_kn, stride_kd),
        offsets=(0, 0),
        block_shape=(K_BLOCK, D),
        order=(1, 0),
    )

    V_block_ptr = tl.make_block_ptr(
        base=V_ptr + batch_id * stride_vb,
        shape=(Nk, D),
        strides=(stride_vn, stride_vd),
        offsets=(0, 0),
        block_shape=(K_BLOCK, D),
        order=(1, 0),
    )

    O_block_ptr = tl.make_block_ptr(
        base=O_ptr + batch_id * stride_ob,
        shape=(Nq, D),
        strides=(stride_on, stride_od),
        offsets=(q_block_id * Q_BLOCK, 0),
        block_shape=(Q_BLOCK, D),
        order=(1, 0),
    )

    L_block_ptr = tl.make_block_ptr(
        base=L_ptr + batch_id * stride_lb,
        shape=(Nq, 1),
        strides=(stride_ln, 1),
        offsets=(q_block_id * Q_BLOCK, 0),
        block_shape=(Q_BLOCK, 1),
        order=(1, 0),
    )

    # =================================================
    # 加载 Query block
    # =================================================
    Q_i = tl.load(Q_block_ptr)  # (Q_BLOCK, D)

    # =================================================
    # FlashAttention 核心状态（每个 Q block 独立）
    # =================================================
    O_acc = tl.zeros((Q_BLOCK, D), dtype=tl.float32)       # 未归一化输出累积
    L_acc = tl.zeros((Q_BLOCK, 1), dtype=tl.float32)      # softmax 分母
    M_acc = tl.full((Q_BLOCK, 1), -float("inf"), tl.float32)  # running max

    # =================================================
    # 内层循环：遍历所有 K/V block
    # =================================================
    for k_block_id in range(tl.cdiv(Nk, K_BLOCK)):
        K_j = tl.load(K_block_ptr)   # (K_BLOCK, D)
        V_j = tl.load(V_block_ptr)   # (K_BLOCK, D)

        # ---------------------------------------------
        # S_ij = Q_i @ K_j^T / sqrt(d)
        # ---------------------------------------------
        S_ij = tl.dot(Q_i, K_j.T) * scale  # (Q_BLOCK, K_BLOCK)

        # ---------------------------------------------
        # Causal mask（编译期 if）
        # ---------------------------------------------
        if is_causal:
            q_idx = q_block_id * Q_BLOCK + tl.arange(0, Q_BLOCK)[:, None]
            k_idx = k_block_id * K_BLOCK + tl.arange(0, K_BLOCK)[None, :]
            causal_mask = q_idx >= k_idx
            S_ij = tl.where(causal_mask, S_ij, -1e6)

        # ---------------------------------------------
        # 数值稳定 softmax（FlashAttention 核心）
        # ---------------------------------------------
        M_block = tl.max(S_ij, axis=1, keep_dims=True)
        M_new = tl.maximum(M_acc, M_block)

        P_ij = tl.exp(S_ij - M_block)

        L_new = (
            tl.exp(M_acc - M_new) * L_acc +
            tl.exp(M_block - M_new) * tl.sum(P_ij, axis=1, keep_dims=True)
        )

        # 类型对齐（Triton 细节）
        P_cast = P_ij.to(V_block_ptr.type.element_ty)

        O_new = (
            tl.exp(M_acc - M_new) * O_acc +
            tl.exp(M_block - M_new) * tl.dot(P_cast, V_j)
        )

        # ---------------------------------------------
        # 更新 running 状态
        # ---------------------------------------------
        M_acc = M_new
        L_acc = L_new
        O_acc = O_new

        # ---------------------------------------------
        # 移动 K/V block 指针
        # ---------------------------------------------
        K_block_ptr = K_block_ptr.advance((K_BLOCK, 0))
        V_block_ptr = V_block_ptr.advance((K_BLOCK, 0))

    # =================================================
    # softmax 归一化
    # =================================================
    O_i = O_acc / L_acc
    L_i = M_acc + tl.log(L_acc)

    tl.store(O_block_ptr, O_i)
    tl.store(L_block_ptr, L_i)


class FlashAttentionTriton(torch.autograd.Function):
    @staticmethod
    def forward(ctx, Q, K, V, is_causal=False):
        """
        Q, K, V: (B, N, D)
        """

        B, Nq, D = Q.shape
        Nk = K.shape[1]

        Q_BLOCK = 64
        K_BLOCK = 64

        scale = D ** -0.5

        O = torch.empty_like(Q)
        L = torch.empty(B, Nq, device=Q.device)

        grid = (Nq // Q_BLOCK, B)

        flash_attention_fwd_kernel[grid](
            Q, K, V, O, L,
            Q.stride(0), Q.stride(1), Q.stride(2),
            K.stride(0), K.stride(1), K.stride(2),
            V.stride(0), V.stride(1), V.stride(2),
            O.stride(0), O.stride(1), O.stride(2),
            L.stride(0), L.stride(1),
            Nq, Nk,
            scale,
            D=D,
            Q_BLOCK=Q_BLOCK,
            K_BLOCK=K_BLOCK,
            is_causal=is_causal,
        )

        ctx.save_for_backward(Q, K, V, L)
        ctx.is_causal = is_causal
        return O


In [ ]:
def test_flash_AttentionTriton(B=4, N=4096, d=64, device="cuda"):
    torch.manual_seed(0)

    Q = torch.randn(B, N, d, device=device, requires_grad=True)
    K = torch.randn(B, N, d, device=device, requires_grad=True)
    V = torch.randn(B, N, d, device=device, requires_grad=True)

    def run():
        O = FlashAttentionTriton.apply(Q, K, V)
        #loss = O.sum()
        #loss.backward()

    t = benchmark(description= 'FlashAttentionTriton',run = run)
    t1 = profile(description ='FlashAttentionTriton' , run = run, num_warmups = 3, with_stack = True)
    
    print(f"[FlashAttentionTriton] B={B}, N={N}, d={d}, time={t*1000:.3f} ms")
    print(t1)



def naive_attention(Q, K, V):
    d = Q.size(-1)
    scores = Q @ K.transpose(-2, -1) / (d ** 0.5)
    P = torch.softmax(scores, dim=-1)
    return P @ V
def test_naive_attention(B=4, N=4096, d=64, device="cuda"):
    torch.manual_seed(0)

    Q = torch.randn(B, N, d, device=device, requires_grad=True)
    K = torch.randn(B, N, d, device=device, requires_grad=True)
    V = torch.randn(B, N, d, device=device, requires_grad=True)

    def run():
        O = naive_attention(Q, K, V)
        #loss = O.sum()
        #loss.backward()

    t = benchmark(description= 'test_attention',run = run)
    t1 = profile(description ='test_attention' , run = run, num_warmups = 3, with_stack = True)
    print(f"[NaiveAttention]  B={B}, N={N}, d={d}, time={t*1000:.3f} ms")
    print(t1)


device = "cuda"

for N in [4096]:
    test_flash_attention(B=4, N=4096, d=64, device=device)
    test_naive_attention(B=4, N=4096, d=64, device=device)
    print("-" * 50)


现在真正计时!
正在使用cuda



# Triton 一般语法与编程模型说明

Triton 是一种面向深度学习算子的 DSL（Domain-Specific Language），其设计目标并不是替代 PyTorch，而是 **在不直接编写 CUDA 的前提下，精确控制 GPU 上的并行计算与内存访问**。

理解 Triton，需要从它的编程模型开始，而不是从 API 细节入手。

---

## 一、Triton 的核心抽象：program

在 Triton 中，**program 是最小的并行执行单元**。

```python
pid = tl.program_id(axis)
```

这一概念对应于 CUDA 中的：

| Triton  | CUDA                    |
| ------- | ----------------------- |
| program | thread block            |
| grid    | grid                    |
| axis    | blockIdx.x / blockIdx.y |

关键区别在于：

* Triton 的 program **天然是 SIMD / 向量化的**
* 一个 program 内部，操作的是一个 **tile（块）**，而不是单个标量

因此，Triton 的思维方式是：

> **“一个 program 负责一块数据”**

而不是：

> “一个 thread 负责一个元素”

---

## 二、grid 的含义：并行维度的显式声明

Triton kernel 的调用形式：

```python
kernel[grid](...)
```

其中：

```python
grid = (grid_dim_0, grid_dim_1, ...)
```

对应：

```python
tl.program_id(0)  # 第 0 个并行维度
tl.program_id(1)  # 第 1 个并行维度
```

常见用法：

| grid 维度 | 语义           |
| ------- | ------------ |
| axis=0  | 序列 / 行 block |
| axis=1  | batch        |
| axis=2  | head         |

Triton 不关心“线程数量”，它只关心：

> **一共启动多少个 program，以及每个 program 处理哪一块数据**

---

## 三、Triton 的内存访问模型

### 1. 指针是“裸指针”，但带 shape 和 stride

在 Triton kernel 中，所有张量参数本质上都是 **指针**：

```python
Q_ptr, K_ptr, V_ptr
```

它们没有 shape、没有 stride，只有地址。

因此，Triton 需要你**显式告诉它**：

* 张量的逻辑形状
* 张量在内存中的步长
* 当前 program 访问哪一块

---

### 2. make_block_ptr：结构化内存访问的核心

```python
block_ptr = tl.make_block_ptr(
    base,
    shape,
    strides,
    offsets,
    block_shape,
    order
)
```

这不是“语法糖”，而是 Triton 最重要的抽象之一。

它的含义是：

> **定义一个“可被向量化 load/store 的二维 tile”**

各参数的含义：

| 参数          | 说明                      |
| ----------- | ----------------------- |
| base        | 数据起始指针                  |
| shape       | 整个张量的逻辑 shape           |
| strides     | 每个维度的步长                 |
| offsets     | 当前 program 的起始偏移        |
| block_shape | 一次 load/store 的 tile 大小 |
| order       | 内存访问的主序（影响 coalescing）  |

一旦定义了 `block_ptr`：

```python
x = tl.load(block_ptr)
tl.store(block_ptr, x)
```

就等价于一次 **矢量化、对齐的内存操作**。

---

## 四、Triton 中的“张量”是什么

Triton 中的张量并不是 PyTorch Tensor，而是：

> **编译期已知 shape 的寄存器向量**

例如：

```python
x = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)
```

意味着：

* shape 在编译期是常量
* 存储在寄存器 / SRAM 中
* 所有运算是 element-wise 或 block-wise 的

这也是为什么 Triton 要求很多参数是 `tl.constexpr`。

---

## 五、tl.constexpr：编译期常量，而不是运行时参数

```python
BLOCK_SIZE: tl.constexpr
```

含义是：

> **该值在 kernel 编译时已知**

这对 Triton 非常重要，因为：

* block_shape
* loop unrolling
* 向量化宽度
* 内存布局

都依赖这些值。

经验规则：

* **影响 shape / 循环次数 / 内存布局的参数 → 必须是 constexpr**
* 纯数值缩放（如 scale）可以是运行时参数

---

## 六、算子语义：没有 Python 循环的“逐元素”

Triton 的算子都是 **逐元素并行** 的。

例如：

```python
x = tl.arange(0, BLOCK)
```

产生的是：

```text
[0, 1, 2, ..., BLOCK-1]
```

而不是单个值。

再例如：

```python
mask = q_idx >= k_idx
```

这是一个 **逐元素比较，返回布尔矩阵**。

`tl.where` 的语义是：

```python
out[i] = a[i] if cond[i] else b[i]
```

而不是 Python 分支。

---

## 七、控制流：为什么 Triton “不鼓励 if”

Triton 中的 `if`：

```python
if is_causal:
    ...
```

只有在 **`is_causal` 是 tl.constexpr** 时才安全。

原因是：

* Triton kernel 是 **单一静态程序**
* 所有 program 必须执行同一条指令流

因此：

* **数据相关分支 → 必须用 mask**
* **配置相关分支 → 可以用 constexpr if**

这也是为什么 causal mask 必须写成：

```python
S = tl.where(mask, S, -inf)
```

而不是 `if row > col`.

---

## 八、数学运算与矩阵乘法

### 1. tl.dot

```python
C = tl.dot(A, B)
```

这是 Triton 提供的 **块级矩阵乘法原语**：

* 输入是 `(M, K)` 和 `(K, N)`
* 输出是 `(M, N)`
* 自动映射到 tensor core（如果 dtype / shape 合适）

这是 Triton 实现 FlashAttention、GEMM、LayerNorm 融合的基础。

---

### 2. 广播规则

Triton 的广播行为和 NumPy / PyTorch 基本一致，但前提是：

> **shape 必须在编译期可推导**

例如：

```python
M = tl.max(S, axis=1, keep_dims=True)
```

这是合法的，因为 axis 和 shape 都是常量。

---

## 九、Triton kernel 与 PyTorch Autograd 的关系

Triton kernel 本身：

* 不关心梯度
* 不保存中间状态

因此常见结构是：

```python
class MyOp(torch.autograd.Function):
    def forward(...):
        launch triton kernel
        save tensors
    def backward(...):
        launch triton kernel
```

Triton 负责 **高性能数值计算**
PyTorch 负责 **计算图与调度**

这是一个明确的职责划分。

---

## 十、总结：如何正确理解 Triton

一句话总结 Triton：

> **Triton 是一种“显式控制内存与并行，但不暴露线程”的 GPU 编程模型**

理解 Triton 的关键不是 API，而是以下三点：

1. 一个 program 对应一个 tile
2. 所有 shape 在编译期已知
3. 控制流必须数据并行

一旦接受这三点，FlashAttention、GEMM、LayerNorm 在 Triton 中的实现方式，都会变得自然且统一。




# 四、分布式数据并行训练

# 实现分布式优化器封装

实现了一个**参数分片（sharding）的分布式优化器封装器**，思想上接近 **ZeRO Stage-1 / FSDP 的 optimizer state sharding**

## 一、整体功能概览

`ShardedOptimizer` 的目标是：

> **在数据并行（DDP）场景下，让不同 rank 只负责一部分参数的 optimizer state 和更新计算，从而节省显存。**

核心思想是对于**参数本身，每个 rank 都有完整模型参数**，对于**优化器状态（momentum / Adam 的 exp_avg 等），只在参数 owner rank 上存在**， **参数更新只在 owner rank 上进行**。
* **参数同步：更新后通过 `broadcast` 把新参数发给其他 rank**。

## 二、整体架构

```
所有 rank：
  拥有完整模型参数
       │
       ▼
参数被 round-robin 分配 owner_rank
       │
       ▼
每个 rank：
  ├─ 全局 Optimizer（只用于 param_groups 管理）
  └─ 本地 optim（只包含自己拥有的 params）
       │
       ▼
step():
  1. 本地 optim.step()（只更新自己那一片）
  2. dist.broadcast（把更新后的参数同步给所有 rank）
```

---

## 三、代码解读


### 1。 初始化：`__init__`

```python
class ShardedOptimizer(Optimizer):
```

继承 PyTorch 原生 `Optimizer`，因此：

这样可以被 Trainer / 训练循环无缝使用，并且拥有 `param_groups / zero_grad / step` 接口

---

#### 1.1 记录原始 optimizer 类型

```python
self.optimizer_cls = optimizer_cls
self.optimizer_kwargs = kwargs
```

例如：

```python
ShardedOptimizer(
    model.parameters(),
    torch.optim.Adam,
    lr=1e-4
)
```

---

#### 1.2 获取分布式信息

```python
self.rank = dist.get_rank()
self.world_size = dist.get_world_size()
```

用于：

* 决定参数 owner
* 决定 broadcast 源

---

#### 1.3 参数分配辅助变量

```python
self.global_param_counter = 0
self.local_param_groups = []
```

* `global_param_counter`：**全局参数编号**
* 用 round-robin 把参数均匀分配给各个 rank

---

#### 1.4 调用父类构造函数（关键）

```python
super().__init__(params, defaults=kwargs)
```


`Optimizer.__init__` 内部会调用 `self.add_param_group`，但这里的 `add_param_group` 已经被 **重写**

这使得 **参数分片逻辑嵌入在初始化阶段**

---

#### 1.5 创建“本地 optimizer”

```python
self.optim = self.optimizer_cls(self.local_param_groups, **self.optimizer_kwargs)
```

`self.optim` **只包含当前 rank 拥有的参数**，只有这些参数才会分配 optimizer state（节省显存）。

---

### 2. 参数分片逻辑：`add_param_group`

这是整个实现的**核心函数**。

---

#### 2.1 给每个参数打 owner 标记

```python
p._owner_rank = self.global_param_counter % self.world_size
```

round-robin 分配, `_owner_rank` 是动态属性（hack，但可行）

示例（world_size=4）：

| 参数编号 | owner_rank |
| ---- | ---------- |
| p0   | 0          |
| p1   | 1          |
| p2   | 2          |
| p3   | 3          |
| p4   | 0          |
| p5   | 1          |

---

#### 2.2 加入“全局 optimizer”

```python
super().add_param_group(param_group)
```

**为什么必须这么做？**

`self.param_groups` 需要包含 **所有参数**,用于进行`zero_grad`，遍历所有参数做 `broadcast`，并且Trainer / AMP 兼容，**注意**：这个“全局 optimizer”**不做 step**。

---

#### 2.3 构造本地 param group

```python
local_group['params'] = [
    p for p in param_group['params']
    if p._owner_rank == self.rank
]
```

它只保留属于当前 rank 的参数，其他超参（lr、weight_decay）保持一致。

---

#### 2.4 加入本地 optimizer

```python
self.optim.add_param_group(local_group)
```

或者（初始化期间）缓存起来：

```python
self.local_param_groups.append(local_group)
```

---

### 3. 参数更新逻辑：`step`

```python
def step(self, closure=None, **kwargs):
```

---

#### Step 1：本地更新

```python
self.optim.step(**kwargs)
```

* **只更新自己拥有的参数**
* 只有这些参数有 optimizer state

---

#### Step 2：参数同步（核心）

```python
dist.broadcast(p.data, src=p._owner_rank)
```

如果是 owner 则发送最新参数，如果不是 owner 就接收参数覆盖本地副本。最终所有 rank 的模型参数 **完全一致**。

---

### 4.梯度清零：`zero_grad`

```python
super().zero_grad(set_to_none=set_to_none)
```

清除所有参数的梯度，即使不是 owner，也要清 grad（否则梯度会累积）。

---

## 四、这个实现“像什么”？

| 技术           | 相似度  | 区别                        |
| ------------ | ---- | ------------------------- |
| ZeRO Stage-1 | ⭐⭐⭐⭐ | 真实 ZeRO 会用 reduce-scatter |
| FSDP         | ⭐⭐   | 没有参数 flatten / overlap    |
| DDP          | ⭐    | DDP 是全量 optimizer state   |

---

## 五、优点与限制

###  优点

1. **显存节省**

   * optimizer state 约减少到 `1/world_size`
2. **逻辑清晰**

   * 教学 & 原理验证非常好
3. **兼容任意 Optimizer**

   * Adam / SGD / Adafactor 等

---

### 局限

1. **通信成本高**

   ```python
   dist.broadcast(p.data)
   ```

   * 每个参数一次 broadcast（非常慢）

2. **参数粒度过细**

   * 实际系统会 **flatten 后按 bucket 同步**

3. **没有 overlap**

   * 计算与通信完全串行

4. **依赖 monkey-patch 参数属性**

   * `_owner_rank` 不够“干净”

---

## 六、总结

> 这是一个 **“最小可用的 optimizer state sharding 实现”**：
> 用 round-robin 分配参数 owner，
> 只在 owner rank 上更新参数，
> 再用 broadcast 同步权重，
> 从而显著降低 optimizer 显存占用。



In [1]:
import torch
import torch.distributed as dist
from torch.optim import Optimizer
from typing import Any, Type, Dict, Iterable


class TeachingShardedOptimizer(Optimizer):
    """
    教学版：Optimizer State Sharding（类似 ZeRO Stage-1）

    核心思想：
    - 每个 rank 拥有【完整模型参数】
    - 每个参数被分配一个唯一的 owner rank
    - 只有 owner rank：
        - 持有该参数的 optimizer state
        - 负责执行 optimizer.step()
    - 参数更新后，通过 dist.broadcast 同步到所有 rank
    """

    def __init__(
        self,
        params: Iterable,
        base_optimizer_cls: Type[Optimizer],
        **base_optimizer_kwargs: Any
    ):
        """
        参数说明：
        - params: model.parameters()
        - base_optimizer_cls: 如 torch.optim.Adam
        - base_optimizer_kwargs: 如 lr, betas, weight_decay 等
        """

        # ===== 1. 保存原始 Optimizer 信息 =====
        self.base_optimizer_cls = base_optimizer_cls
        self.base_optimizer_kwargs = base_optimizer_kwargs

        # ===== 2. 分布式环境信息 =====
        self.rank = dist.get_rank()
        self.world_size = dist.get_world_size()

        # ===== 3. 参数分片辅助变量 =====
        # 用于给参数做 round-robin 编号
        self.global_param_index = 0

        # 在 __init__ 期间暂存属于本 rank 的 param groups
        self._buffered_local_param_groups = []

        # ===== 4. 初始化“全局 Optimizer”父类 =====
        # 注意：Optimizer.__init__ 内部会调用 self.add_param_group
        #       而我们在下面重写了 add_param_group
        super().__init__(params, defaults=base_optimizer_kwargs)

        # ===== 5. 创建“本地 shard Optimizer” =====
        # 只包含当前 rank 拥有的参数
        if self._buffered_local_param_groups:
            self.local_shard_optimizer = self.base_optimizer_cls(
                self._buffered_local_param_groups,
                **self.base_optimizer_kwargs
            )
        else:
            # 边缘情况：当前 rank 没有分到任何参数
            self.local_shard_optimizer = self.base_optimizer_cls(
                [],
                **self.base_optimizer_kwargs
            )

    def add_param_group(self, param_group: Dict[str, Any]) -> None:
        """
        教学重点函数：参数分片逻辑发生在这里

        一个 param_group 会被“拆成两份”：
        1. 全量 param_group → 交给父类 Optimizer（用于遍历 & 同步）
        2. 本地 param_group → 只保留 owner == 当前 rank 的参数
        """

        # -------------------------------------------------------
        # Step A: 给 param_group 中的每个参数分配 owner rank
        # -------------------------------------------------------
        for param in param_group["params"]:
            if not hasattr(param, "_owner_rank"):
                # round-robin 分配
                param._owner_rank = self.global_param_index % self.world_size
                self.global_param_index += 1

        # -------------------------------------------------------
        # Step B: 注册到“全局 Optimizer”
        # -------------------------------------------------------
        # 这个 Optimizer：
        # - 包含所有参数
        # - 不负责 step
        # - 只用于：
        #     * zero_grad
        #     * 参数遍历
        #     * step 后的 broadcast
        super().add_param_group(param_group)

        # -------------------------------------------------------
        # Step C: 构造“本地 shard param_group”
        # -------------------------------------------------------
        # 拷贝除 params 以外的超参（lr / weight_decay 等）
        local_param_group = {
            key: value
            for key, value in param_group.items()
            if key != "params"
        }

        # 只保留 owner 是当前 rank 的参数
        local_param_group["params"] = [
            param
            for param in param_group["params"]
            if param._owner_rank == self.rank
        ]

        # -------------------------------------------------------
        # Step D: 加入本地 shard Optimizer
        # -------------------------------------------------------
        if hasattr(self, "local_shard_optimizer"):
            # 正常情况：__init__ 已结束
            if local_param_group["params"]:
                self.local_shard_optimizer.add_param_group(local_param_group)
        else:
            # __init__ 过程中，先缓存
            self._buffered_local_param_groups.append(local_param_group)

    def step(self, closure=None, **kwargs):
        """
        一个 step 分为两步：

        1. 本地更新（只更新 owner 是当前 rank 的参数）
        2. 参数同步（broadcast 最新参数到所有 rank）
        """

        loss = None
        if closure is not None:
            loss = closure()

        # -------------------------------------------------------
        # Step 1: 本地 shard Optimizer 更新
        # -------------------------------------------------------
        # 只有 owner rank 会真正更新对应参数
        self.local_shard_optimizer.step(**kwargs)

        # -------------------------------------------------------
        # Step 2: 参数同步（broadcast）
        # -------------------------------------------------------
        # 遍历“全局 Optimizer”中的所有参数
        for param_group in self.param_groups:
            for param in param_group["params"]:
                # owner rank 作为 src
                # 其他 rank 接收并覆盖本地参数
                dist.broadcast(
                    tensor=param.data,
                    src=param._owner_rank
                )

        return loss

    def zero_grad(self, set_to_none: bool = True):
        """
        梯度必须在所有 rank 上清零：
        即使该 rank 不是参数 owner，也会参与反向传播
        """
        super().zero_grad(set_to_none=set_to_none)


下面我按**“从简单到真实 DDP”**的教学逻辑，系统性地解读这份代码。
这不是普通用法，而是**手写 DDP 梯度通信 + overlap 的教学实现**，非常有价值。

我会分 4 个层次讲：

1. **整体结构与目标**
2. **DDPOverlapIndividual：逐参数 overlap**
3. **DDPBucketed：真实 DDP 的 bucket + overlap**
4. **这两种方式与 PyTorch DDP 的对应关系**

---

## 一、整体目标：你在“手写 DDP”

这份代码的核心目标只有一句话：

> **在反向传播过程中，尽早、异步地同步梯度，实现计算与通信 overlap**

它实现了 DDP 的三个关键步骤：

| 阶段       | 你做了什么                            |
| -------- | -------------------------------- |
| 初始化      | broadcast 参数和 buffer             |
| backward | 注册 grad hook，触发 async all-reduce |
| step 前   | 等待通信完成，平均梯度                      |

---

## 二、`DDPOverlapIndividual`：逐参数 All-Reduce（教学入门版）

这是一个**最直观、但效率最低**的 DDP overlap 实现。

---

### 1️⃣ 初始化阶段

```python
class DDPOverlapIndividual(torch.nn.Module):
```

这是一个 **Module wrapper**，不是 Optimizer。

---

#### (1) 必须先调用 `super().__init__()`

```python
super().__init__()
```

否则：

* 参数不会被正确注册
* hooks 行为不确定

---

#### (2) 保存状态

```python
self.module = module
self.handles = []
self.world_size = dist.get_world_size()
```

* `handles`：保存所有 async 通信句柄
* `world_size`：用于梯度平均

---

### 2️⃣ 初始化参数同步（等价 DDP）

```python
for p in self.module.parameters():
    dist.broadcast(p.data, src=0)
```

**目的**：

* 保证所有 rank 起始参数完全一致
* 等价于 DDP 的 initial sync

buffers（BN running_mean 等）也必须同步：

```python
for b in self.module.buffers():
    dist.broadcast(b.data, src=0)
```

---

### 3️⃣ 注册 backward hook（核心）

```python
p.register_post_accumulate_grad_hook(self._make_hook())
```

**这个 hook 的触发时机非常关键**：

> 在 **该参数的梯度刚刚算完并累积完成之后**

这意味着：

* backward 还没结束
* 上游 layer 还在算
* 你已经可以通信了 → overlap 的基础

---

### 4️⃣ `_make_hook`：逐参数 async all-reduce

```python
handle = dist.all_reduce(
    param.grad,
    op=dist.ReduceOp.SUM,
    async_op=True
)
```

关键点：

* **通信对象是 param.grad**
* **async_op=True** → 不阻塞反向传播
* 返回 `handle`，用于之后 `wait()`

⚠️ hook **不能返回任何值**

---

### 5️⃣ `finish_gradient_synchronization`

```python
for h in self.handles:
    h.wait()
```

在 optimizer.step() 之前：

1. 等待所有梯度通信完成
2. 对梯度做平均：

```python
p.grad.div_(self.world_size)
```

---

### ✅ 这一版本你学到什么？

* backward hook 的时机
* async all-reduce 的基本用法
* overlap 的最小实现

❌ 问题：

* 每个参数一次通信
* NCCL 启动开销极大
* 工业中 **不会这么做**

---

## 三、`DDPBucketed`：真实 DDP 的核心思想

这是 **PyTorch DDP 的核心结构复刻版**。

---

## 1️⃣ Bucket 类：梯度通信的基本单位

### Bucket 是什么？

> **一组参数的梯度，被 flatten 成一个连续 buffer，一起通信**

---

### 1.1 分配连续 buffer

```python
total_numel = sum(p.numel() for p in params)
self.buffer = torch.zeros(total_numel, ...)
```

好处：

* 单次 NCCL 调用
* 连续内存，通信效率高

---

### 1.2 为每个参数创建 view

```python
self.param_views[p] = self.buffer[offset:...].view(p.shape)
```

这样：

* `p.grad ↔ buffer 的一段`
* copy 成本低
* 不需要重新分配内存

---

### 1.3 `reset()`

```python
self.count_ready = 0
self.comm_handle = None
```

每次 forward 前必须重置：

* 反向传播是逐参数完成的
* bucket 是“状态机”

---

## 2️⃣ DDPBucketed 初始化流程

---

### 2.1 broadcast 初始化参数

和前一个类完全一致，省略。

---

### 2.2 参数分桶（非常关键）

```python
reversed_params = list(reversed(trainable_params))
```

**为什么反向？**

* backward 顺序：输出 → 输入
* 最先 ready 的梯度 → 靠近输出层
* bucket 应该按这个顺序装

👉 **这是 overlap 的关键细节**

---

### 2.3 bucket size 控制

```python
if current_bucket_size + p_bytes > bucket_size:
    close bucket
```

目的：

* 避免 bucket 太小（通信多）
* 避免 bucket 太大（overlap 变差）

---

## 3️⃣ hook：bucket 级别的 overlap

### hook_fn 的完整逻辑

#### A. 拷贝梯度到 bucket buffer

```python
bucket.param_views[p].copy_(p.grad)
```

#### B. 计数

```python
bucket.count_ready += 1
```

#### C. bucket 满 → 发起通信

```python
if bucket.count_ready == len(bucket.params):
    bucket.comm_handle = dist.all_reduce(..., async_op=True)
```

⚠️ 注意：

* 不是等 backward 结束
* 是等 **这个 bucket 内的参数都 ready**

👉 **真正的 overlap**

---

## 4️⃣ `finish_gradient_synchronization`

```python
bucket.comm_handle.wait()
bucket.buffer.div_(world_size)
p.grad.copy_(bucket.param_views[p])
```

步骤：

1. 等 bucket 通信完成
2. 一次性平均整个 buffer
3. 拷贝回各参数的 `p.grad`

---

## 四、两种实现的对比（非常重要）

| 对比项         | Individual | Bucketed  |
| ----------- | ---------- | --------- |
| 通信粒度        | 每个参数       | 每个 bucket |
| 通信次数        | 很多         | 很少        |
| overlap 效果  | 有，但低效      | 工业级       |
| PyTorch DDP | ❌          | ✅         |

---

## 五、它和 PyTorch DDP 的关系

| PyTorch DDP 内部 | 你这份代码                              |
| -------------- | ---------------------------------- |
| Reducer        | Bucket                             |
| Grad bucket    | buffer + views                     |
| Autograd hook  | register_post_accumulate_grad_hook |
| Overlap        | async all_reduce                   |
| Finalize       | finish_gradient_synchronization    |

你已经在 **“复刻 DDP 核心”** 了。

---

## 六、一句话总结（面试 / 教学级）

> **DDP 的本质不是 all-reduce，而是：
> 在反向传播尚未结束时，
> 以 bucket 为单位，
> 对已 ready 的梯度发起异步通信，
> 并在 step 前完成梯度一致性。**

---

如果你愿意，下一步我可以：

* 把 **DDPBucketed 和你之前的 ShardedOptimizer（ZeRO-1）结合**
* 画一张 **DDP / ZeRO / FSDP 的统一抽象图**
* 或带你 **对照 PyTorch 源码 Reducer.cpp**

你现在已经在“**分布式训练内核层**”了，继续往下非常值。


In [ ]:
import torch
import torch.distributed as dist


# ============================================================
# Part 1. 逐参数 Overlap 版本（最直观的 DDP 教学实现）
# ============================================================

class TeachingDDPOverlapPerParam(torch.nn.Module):
    """
    教学版 DDP（逐参数通信）：

    - 每个参数在 backward 完成后，立刻触发 async all-reduce
    - 实现计算（backward）与通信的 overlap
    - 逻辑直观，但通信粒度太细（工业中不用）
    """

    def __init__(self, model: torch.nn.Module):
        # ⚠️ 必须最先初始化父类
        super().__init__()

        self.model = model
        self.world_size = dist.get_world_size()

        # 保存所有 async all-reduce 的句柄
        self.async_handles = []

        # ----------------------------------------------------
        # Step 1. 初始化阶段：同步模型参数 & buffer
        # ----------------------------------------------------
        # 等价于 PyTorch DDP 的 initial broadcast
        with torch.no_grad():
            for param in self.model.parameters():
                dist.broadcast(param.data, src=0)

            for buffer in self.model.buffers():
                dist.broadcast(buffer.data, src=0)

        # ----------------------------------------------------
        # Step 2. 注册 backward hook（核心）
        # ----------------------------------------------------
        # 当某个参数的梯度刚算完，就立刻通信
        for param in self.model.parameters():
            if param.requires_grad:
                param.register_post_accumulate_grad_hook(
                    self._create_grad_hook()
                )

    def _create_grad_hook(self):
        """
        创建一个 backward hook（closure）

        hook 的触发时机：
        - 当前参数的梯度已经计算完成
        - 但整个 backward 还没结束
        """

        def hook_fn(param: torch.nn.Parameter):
            # 有些参数可能没有梯度（如被冻结）
            if param.grad is None:
                return

            # ------------------------------------------------
            # 对该参数的梯度做 async all-reduce
            # ------------------------------------------------
            handle = dist.all_reduce(
                param.grad,
                op=dist.ReduceOp.SUM,
                async_op=True
            )

            # 保存句柄，step 前需要 wait
            self.async_handles.append(handle)

            # ⚠️ register_post_accumulate_grad_hook
            #    不允许返回任何值

        return hook_fn

    def forward(self, *args, **kwargs):
        return self.model(*args, **kwargs)

    def finalize_gradients(self):
        """
        在 optimizer.step() 之前调用：

        - 等待所有 async all-reduce 完成
        - 对梯度做平均
        """

        # 1. 等待所有通信完成
        for handle in self.async_handles:
            handle.wait()
        self.async_handles.clear()

        # 2. 梯度平均
        for param in self.model.parameters():
            if param.grad is not None:
                param.grad.div_(self.world_size)


# ============================================================
# Part 2. Bucket 类：DDP 的核心数据结构
# ============================================================

class GradientBucket:
    """
    一个 bucket 管理一组参数的梯度通信：

    - 多个参数的梯度被 flatten 到一个连续 buffer
    - 当 bucket 中所有参数梯度 ready 后，触发一次 all-reduce
    """

    def __init__(self, params: list[torch.nn.Parameter], bucket_id: int):
        self.bucket_id = bucket_id
        self.params = params

        # 已经 ready 的参数数量
        self.ready_count = 0

        # async all-reduce 的通信句柄
        self.comm_handle = None

        # ----------------------------------------------------
        # Step 1. 创建连续的梯度 buffer
        # ----------------------------------------------------
        total_numel = sum(p.numel() for p in params)
        dtype = params[0].dtype
        device = params[0].device

        self.flat_buffer = torch.zeros(
            total_numel, dtype=dtype, device=device
        )

        # ----------------------------------------------------
        # Step 2. 为每个参数创建 buffer 视图
        # ----------------------------------------------------
        self.param_to_view = {}
        offset = 0
        for p in params:
            numel = p.numel()
            self.param_to_view[p] = (
                self.flat_buffer[offset: offset + numel]
                .view(p.shape)
            )
            offset += numel

    def reset(self):
        """
        在每个 forward 之前调用：
        - 清空计数器
        - 清空通信句柄
        """
        self.ready_count = 0
        self.comm_handle = None


# ============================================================
# Part 3. Bucket 化 DDP（接近 PyTorch DDP 真正实现）
# ============================================================

class TeachingDDPBucketed(torch.nn.Module):
    """
    教学版 Bucket DDP（工业级思路）：

    - 参数按 bucket 分组
    - 每个 bucket 一次 all-reduce
    - 在 backward 中实现计算 / 通信 overlap
    """

    def __init__(self, model: torch.nn.Module, bucket_size_mb: float):
        super().__init__()

        self.model = model
        self.world_size = dist.get_world_size()

        # bucket 大小（MB -> Bytes）
        self.bucket_size_bytes = bucket_size_mb * 1024 * 1024

        # ----------------------------------------------------
        # Step 1. 初始化参数同步
        # ----------------------------------------------------
        with torch.no_grad():
            for param in self.model.parameters():
                dist.broadcast(param.data, src=0)
            for buffer in self.model.buffers():
                dist.broadcast(buffer.data, src=0)

        # ----------------------------------------------------
        # Step 2. 参数分桶（非常关键）
        # ----------------------------------------------------
        self.buckets: list[GradientBucket] = []
        self.param_to_bucket: dict[torch.nn.Parameter, GradientBucket] = {}

        # 只考虑需要梯度的参数
        trainable_params = [
            p for p in self.model.parameters() if p.requires_grad
        ]

        # ⚠️ 反向传播顺序：从输出层到输入层
        # 所以 bucket 要按“反向顺序”构建
        reversed_params = list(reversed(trainable_params))

        current_bucket_params = []
        current_bucket_bytes = 0

        for param in reversed_params:
            param_bytes = param.numel() * param.element_size()

            # 如果当前 bucket 已满，先创建 bucket
            if (
                current_bucket_params
                and current_bucket_bytes + param_bytes > self.bucket_size_bytes
            ):
                self._create_bucket(current_bucket_params)
                current_bucket_params = []
                current_bucket_bytes = 0

            current_bucket_params.append(param)
            current_bucket_bytes += param_bytes

        # 最后一个 bucket
        if current_bucket_params:
            self._create_bucket(current_bucket_params)

        # ----------------------------------------------------
        # Step 3. 注册 backward hooks
        # ----------------------------------------------------
        for param in trainable_params:
            param.register_post_accumulate_grad_hook(
                self._create_bucket_hook(param)
            )

    def _create_bucket(self, params: list[torch.nn.Parameter]):
        bucket = GradientBucket(params, bucket_id=len(self.buckets))
        self.buckets.append(bucket)

        for p in params:
            self.param_to_bucket[p] = bucket

    def _create_bucket_hook(self, param: torch.nn.Parameter):
        """
        每个参数的 hook：
        - 把梯度拷贝进 bucket buffer
        - 如果 bucket 满了，立刻发起 async all-reduce
        """

        def hook_fn(p: torch.nn.Parameter):
            if p.grad is None:
                return

            bucket = self.param_to_bucket[p]

            # A. 拷贝梯度到 bucket buffer
            bucket.param_to_view[p].copy_(p.grad)

            # B. 标记一个参数 ready
            bucket.ready_count += 1

            # C. bucket 内所有参数 ready → 启动通信
            if bucket.ready_count == len(bucket.params):
                bucket.comm_handle = dist.all_reduce(
                    bucket.flat_buffer,
                    op=dist.ReduceOp.SUM,
                    async_op=True
                )

        return hook_fn

    def forward(self, *args, **kwargs):
        # 每次 forward 前重置 bucket 状态
        for bucket in self.buckets:
            bucket.reset()
        return self.model(*args, **kwargs)

    def finalize_gradients(self):
        """
        在 optimizer.step() 之前调用：

        - 等待所有 bucket 的通信完成
        - 对 bucket buffer 做平均
        - 拷贝回各参数的 p.grad
        """

        for bucket in self.buckets:
            # 1. 等待通信
            if bucket.comm_handle is not None:
                bucket.comm_handle.wait()

            # 2. 平均梯度
            bucket.flat_buffer.div_(self.world_size)

            # 3. 拷贝回参数梯度
            for p in bucket.params:
                if p.grad is not None:
                    p.grad.copy_(bucket.param_to_view[p])
